In [1]:
#!pip3 install pandas
#!pip3 install PyArrow
from pyspark.sql.functions import col
import os

In [2]:
#df = pd.read_csv("train.gz")

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import TimestampType

In [4]:
from IPython.display import Javascript, HTML
import os
import time
import json
import requests
import xml.etree.ElementTree as ET
import datetime
import subprocess

In [5]:
if os.path.exists("/etc/hadoop/conf/hive-site.xml"):
    tree = ET.parse("/etc/hadoop/conf/hive-site.xml")
    root = tree.getroot()
    for prop in root.findall("property"):
        if prop.find("name").text == "hive.metastore.warehouse.dir":
            storage = (
                prop.find("value").text.split("/")[0]
                + "//"
                + prop.find("value").text.split("/")[2]
            )

In [6]:
os.environ["STORAGE"] = storage

In [7]:
print(storage)

s3a://go01-demo


In [8]:
spark = SparkSession\
    .builder\
    .appName("PythonSQL")\
    .config("spark.hadoop.fs.s3a.s3guard.ddb.region","us-east-2")\
    .config("spark.yarn.access.hadoopFileSystems",os.environ["STORAGE"])\
    .config("spark.rpc.message.maxSize", "1024")\
    .config("spark.dynamicAllocation.enabled", "true")\
    .getOrCreate()
 #   .config("spark.driver.cores", 4)\
 #   .config("spark.driver.memory", "8g")\
    

Setting spark.hadoop.yarn.resourcemanager.principal to pauldefusco


In [9]:
df = spark.read.parquet(os.environ["STORAGE"]+'/cde-workshop/clickthrough/data_large_from_csv',   
    header=True,
    sep=',',
    nullValue='NA')

In [9]:
df.take(2)

[Row(id='8.557257391414261e+18', click='0', hour='2014-10-24 07:00:00', C1='1005', banner_pos='1', site_id='e151e245', site_domain='7e091613', site_category='f028772b', app_id='ecad2386', app_domain='7801e8d9', app_category='07d7df22', device_id='a99f214a', device_ip='f73cf001', device_model='a0f5f879', device_type='1', device_conn_type='0', C14='4687', C15='320', C16='50', C17='423', C18='2', C19='39', C20='100148', C21='32'),
 Row(id='3.955540169721846e+18', click='1', hour='2014-10-24 09:00:00', C1='1005', banner_pos='0', site_id='1fbe01fe', site_domain='f3845767', site_category='28905ebd', app_id='ecad2386', app_domain='7801e8d9', app_category='07d7df22', device_id='a99f214a', device_ip='aa92a552', device_model='f07e20f8', device_type='1', device_conn_type='2', C14='20108', C15='320', C16='50', C17='2299', C18='2', C19='1319', C20='100084', C21='52')]

In [19]:
df.take(2)

[Row(id='8.557257391414261e+18', click='0', hour='2014-10-24 07:00:00', C1='1005', banner_pos='1', site_id='e151e245', site_domain='7e091613', site_category='f028772b', app_id='ecad2386', app_domain='7801e8d9', app_category='07d7df22', device_id='a99f214a', device_ip='f73cf001', device_model='a0f5f879', device_type='1', device_conn_type='0', C14='4687', C15='320', C16='50', C17='423', C18='2', C19='39', C20='100148', C21='32'),
 Row(id='3.955540169721846e+18', click='1', hour='2014-10-24 09:00:00', C1='1005', banner_pos='0', site_id='1fbe01fe', site_domain='f3845767', site_category='28905ebd', app_id='ecad2386', app_domain='7801e8d9', app_category='07d7df22', device_id='a99f214a', device_ip='aa92a552', device_model='f07e20f8', device_type='1', device_conn_type='2', C14='20108', C15='320', C16='50', C17='2299', C18='2', C19='1319', C20='100084', C21='52')]

In [20]:
df.rdd.getNumPartitions()

25

In [21]:
df.dtypes

[('id', 'string'),
 ('click', 'string'),
 ('hour', 'string'),
 ('C1', 'string'),
 ('banner_pos', 'string'),
 ('site_id', 'string'),
 ('site_domain', 'string'),
 ('site_category', 'string'),
 ('app_id', 'string'),
 ('app_domain', 'string'),
 ('app_category', 'string'),
 ('device_id', 'string'),
 ('device_ip', 'string'),
 ('device_model', 'string'),
 ('device_type', 'string'),
 ('device_conn_type', 'string'),
 ('C14', 'string'),
 ('C15', 'string'),
 ('C16', 'string'),
 ('C17', 'string'),
 ('C18', 'string'),
 ('C19', 'string'),
 ('C20', 'string'),
 ('C21', 'string')]

In [10]:
#spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")
df = df.withColumn("converted_date",df['hour'].cast(TimestampType()))

In [11]:
df = df.withColumn('year',F.year("converted_date"))\
            .withColumn('month', F.month("converted_date"))\
            .withColumn('day', F.dayofmonth("converted_date"))\
            .withColumn('dt_hour', F.hour("converted_date"))

In [24]:
df.dtypes

[('id', 'string'),
 ('click', 'string'),
 ('hour', 'string'),
 ('C1', 'string'),
 ('banner_pos', 'string'),
 ('site_id', 'string'),
 ('site_domain', 'string'),
 ('site_category', 'string'),
 ('app_id', 'string'),
 ('app_domain', 'string'),
 ('app_category', 'string'),
 ('device_id', 'string'),
 ('device_ip', 'string'),
 ('device_model', 'string'),
 ('device_type', 'string'),
 ('device_conn_type', 'string'),
 ('C14', 'string'),
 ('C15', 'string'),
 ('C16', 'string'),
 ('C17', 'string'),
 ('C18', 'string'),
 ('C19', 'string'),
 ('C20', 'string'),
 ('C21', 'string'),
 ('converted_date', 'timestamp'),
 ('year', 'int'),
 ('month', 'int'),
 ('day', 'int'),
 ('dt_hour', 'int')]

In [25]:
#df.select(F.min('converted_date')).show()

In [26]:
#df.select(F.max('converted_date')).show()

In [12]:
df_original = df.filter(df.converted_date < '2014-10-26 00:00:00')

In [13]:
df_batch1 = df.filter((df.converted_date >= '2014-10-26 00:00:00') & (df.converted_date < '2014-10-27 00:00:00'))

In [14]:
df_batch1_noise = df.filter((df.converted_date >= '2014-10-25 00:00:00') & (df.converted_date < '2014-10-26 00:00:00')).sample(0.1)
#df_original.sample(fraction=0.25, seed=3)

In [15]:
df_original_subtract = df_batch1_noise.sample(.4)

In [16]:
df_original = df_original.subtract(df_original_subtract)

In [17]:
df_batch1 = df_batch1.union(df_batch1_noise)

In [33]:
df_batch2 = df.filter((df.converted_date >= '2014-10-27 00:00:00') & (df.converted_date < '2014-10-28 00:00:00'))

In [34]:
df_batch2_noise1 = df_batch1.sample(0.1)

In [35]:
df_batch1_subtract = df_batch2_noise1.sample(.35)

In [36]:
df_batch1 = df_batch1.subtract(df_batch1_subtract)

In [37]:
df_batch2 = df_batch2.union(df_batch2_noise1)

In [38]:
#df_batch1.count()

In [39]:
df_batch3 = df.filter((df.converted_date >= '2014-10-28 00:00:00') & (df.converted_date < '2014-10-29 00:00:00'))

In [40]:
df_batch3_noise1 = df_batch2.sample(0.09)

In [41]:
df_batch2_subtract = df_batch3_noise1.sample(.45)

In [42]:
df_batch2 = df_batch2.subtract(df_batch2_subtract)

In [43]:
df_batch3 = df_batch3.union(df_batch3_noise1)

In [44]:
#df_batch2.count()

In [45]:
df_batch4 = df.filter((df.converted_date >= '2014-10-29 00:00:00') & (df.converted_date <= '2014-10-30 00:00:00'))

In [46]:
df_batch4_noise1 = df_batch3.sample(0.31)

In [47]:
df_batch3_subtract = df_batch4_noise1.sample(.20)

In [48]:
df_batch3 = df_batch3.subtract(df_batch3_subtract)

In [49]:
df_batch4 = df_batch4.union(df_batch4_noise1)

In [50]:
#df.select(F.max('converted_date')).show()

In [51]:
#df.select(F.min('converted_date')).show()

In [52]:
drop_cols = ['converted_date','year', 'month','day', 'dt_hour']

In [53]:
df_original = df_original.drop(*drop_cols)
df_batch1 = df_batch1.drop(*drop_cols)
df_batch2 = df_batch2.drop(*drop_cols)
df_batch3 = df_batch3.drop(*drop_cols)
df_batch4 = df_batch4.drop(*drop_cols)

In [45]:
df_batch1\
    .write.mode("overwrite").option("header","true")\
    .csv("s3a://go01-demo/cde-workshop/clickthrough/csv_batch_01")

22/10/24 22:22:02 012 ERROR TransportResponseHandler: Still have 1 requests outstanding when connection from /100.100.164.117:42756 is closed


In [46]:
df_batch2\
    .write.mode("overwrite").option("header","true")\
    .csv("s3a://go01-demo/cde-workshop/clickthrough/csv_batch_02")

22/10/24 22:23:18 564 ERROR TransportResponseHandler: Still have 1 requests outstanding when connection from /100.100.164.125:49090 is closed
22/10/24 22:26:32 361 ERROR TransportResponseHandler: Still have 1 requests outstanding when connection from /100.100.164.114:46046 is closed
22/10/24 22:31:54 352 ERROR TransportResponseHandler: Still have 1 requests outstanding when connection from /100.100.210.160:39856 is closed
22/10/24 22:38:56 786 ERROR TransportResponseHandler: Still have 2 requests outstanding when connection from /100.100.164.66:34964 is closed
22/10/24 22:38:56 979 ERROR TransportResponseHandler: Still have 1 requests outstanding when connection from /100.100.137.114:49552 is closed
22/10/24 22:49:09 651 ERROR TransportResponseHandler: Still have 1 requests outstanding when connection from /100.100.210.151:47282 is closed
22/10/24 22:50:55 288 ERROR TransportResponseHandler: Still have 1 requests outstanding when connection from /100.100.164.80:36326 is closed
22/10/24

In [54]:
df_batch3\
    .write.mode("overwrite").option("header","true")\
    .csv("s3a://go01-demo/cde-workshop/clickthrough/csv_batch_03")

In [55]:
df_batch4\
    .write.mode("overwrite").option("header","true")\
    .csv("s3a://go01-demo/cde-workshop/clickthrough/csv_batch_04")

In [19]:
drop_cols = ["converted_date", "year", "month", "day", "dt_hour"]

In [20]:
df_original = df_original.drop(*drop_cols)

In [21]:
df_original.dtypes

[('id', 'string'),
 ('click', 'string'),
 ('hour', 'string'),
 ('C1', 'string'),
 ('banner_pos', 'string'),
 ('site_id', 'string'),
 ('site_domain', 'string'),
 ('site_category', 'string'),
 ('app_id', 'string'),
 ('app_domain', 'string'),
 ('app_category', 'string'),
 ('device_id', 'string'),
 ('device_ip', 'string'),
 ('device_model', 'string'),
 ('device_type', 'string'),
 ('device_conn_type', 'string'),
 ('C14', 'string'),
 ('C15', 'string'),
 ('C16', 'string'),
 ('C17', 'string'),
 ('C18', 'string'),
 ('C19', 'string'),
 ('C20', 'string'),
 ('C21', 'string')]

In [22]:
df_original.write.mode("overwrite").option("header","true").parquet("s3a://go01-demo/cde-workshop/clickthrough/historical")

In [ ]:
#sparkDF.write\
# .option("header","true")\
# .parquet("s3a://go01-demo/cde-workshop/clickthrough")